In [ ]:
!pip install pylabwons

In [1]:
import pylabwons as lw
import os
IS_COLAB = any([key.startswith('COLAB') for key in os.environ])
if IS_COLAB:
    from google.colab import files
else:
    from scripts.env import PATH

# WICS CLASSIFICATION

In [ ]:
wics = lw.WiseICS()
wics.fetch()
# print(wics.logger)
if IS_COLAB:
    wics.to_parquet('wics.parquet', engine='pyarrow')
    files.download('wics.parquet')
else:
    wics.to_parquet(PATH.PARQ.WICS, engine='pyarrow')

# AFTER MARKET

In [4]:
market = lw.AfterMarket()
market.fetch()
# print(market.logger)
if IS_COLAB:
    market.to_parquet('aftermarket.parquet', engine='pyarrow')
    files.download('aftermarket.parquet')
else:
    market.to_parquet(PATH.PARQ.AFTERMARKET, engine='pyarrow')

2026-02-25 15:38:04 FETCH AFTER MARKET DATA ON 20260225
2026-02-25 15:42:30 .............................. 2657 STOCKS / RUNTIME: 265.99s


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# FUNDAMENTAL

In [6]:
base = wics.join(market.drop(columns=['name']))
target = base[base['marketCap'] >= base['marketCap'].median()]
len(target)

1232

In [7]:
fundamental = lw.Fundamentals()
fundamental.fetch(*target.index)
# print(fundamental.logger)
for col in fundamental.columns:
    if fundamental[col].dtype == 'object':
        fundamental[col] = fundamental[col].astype(str)
if IS_COLAB:
    fundamental.to_parquet('fundamentals.parquet', engine='pyarrow')
    files.download('fundamentals.parquet')
else:
    fundamental.to_parquet(PATH.PARQ.FUNDAMENTALS, engine='pyarrow')

0it [00:00, ?it/s]

2026-02-25 15:45:39 FETCH MARKET NUMBERS OF 2026/02/24
2026-02-25 16:08:46 .............................. 1232 STOCKS / RUNTIME: 1388.84s


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>